# Load the libraries

In [1]:
!pip install mysqlclient
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame


In [2]:
csv_file = "Snow_Emergency_Howe_Tows_2018.csv"
howe_tows_df = pd.read_csv(csv_file)
howe_tows_df.head()

,X,Y,FID,Service_Da,Address,Tow_Reason,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day,Snow_Emerg
0,-93.310768,44.985471,1,04/16/2018 02:43pm,"710 RUSSELL AVE N,Minneapolis,MN",Snow Emergency,44.985471,-93.310768,5,Near North,Willard - Hay,1,2,Howe
1,-93.220151,44.971199,2,04/16/2018 02:43pm,"150 26th Ave SE,Minneapolis,MN",Snow Emergency,44.971199,-93.220151,2,University,Prospect Park - East River Road,4,2,Howe
2,-93.262426,45.006359,3,04/16/2018 02:44pm,"360 18th Ave NE,Minneapolis,MN",Snow Emergency,45.006359,-93.262426,1,Northeast,Holland,2,2,Howe
3,-93.227075,44.939345,4,04/16/2018 02:46pm,"3520 31st st e ,Minneapolis,MN",Snow Emergency,44.939345,-93.227075,12,Longfellow,Howe,6,2,Howe
4,-93.294223,45.027867,5,04/16/2018 02:47pm,"4004 Emerson Ave n,Minneapolis,MN",Snow Emergency,45.027867,-93.294223,4,Camden,Webber - Camden,1,2,Howe


### Clean DataFrame

### Transform howe_tows DataFrame

In [3]:
howe_tows_new = howe_tows_df[['FID','Snow_Emerg','Service_Da','Address','Latitude','Longitude','Day','Ward','Neighborho','Community','Tow_Zone']].copy()
howe_tows_new.head()

,FID,Snow_Emerg,Service_Da,Address,Latitude,Longitude,Day,Ward,Neighborho,Community,Tow_Zone
0,1,Howe,04/16/2018 02:43pm,"710 RUSSELL AVE N,Minneapolis,MN",44.985471,-93.310768,2,5,Willard - Hay,Near North,1
1,2,Howe,04/16/2018 02:43pm,"150 26th Ave SE,Minneapolis,MN",44.971199,-93.220151,2,2,Prospect Park - East River Road,University,4
2,3,Howe,04/16/2018 02:44pm,"360 18th Ave NE,Minneapolis,MN",45.006359,-93.262426,2,1,Holland,Northeast,2
3,4,Howe,04/16/2018 02:46pm,"3520 31st st e ,Minneapolis,MN",44.939345,-93.227075,2,12,Howe,Longfellow,6
4,5,Howe,04/16/2018 02:47pm,"4004 Emerson Ave n,Minneapolis,MN",45.027867,-93.294223,2,4,Webber - Camden,Camden,1


In [4]:

# Rename the column headers
howe_tows_transformed = howe_tows_new.rename(columns={'FID':'FID','Snow_Emerg':'snow_emergency_name',
                                                      'Service_Da':'date','Address':'address',
                                                      'Latitude':'latitude','Longitude':'longitude',
                                                      'Day':'day_id','Ward':'ward','Neighborho':'neighborhood',
                                                      'Community':'community','Tow_Zone':'tow_district'})
# Convert date column to datetime

howe_tows_transformed.date = pd.to_datetime(howe_tows_transformed.date)
# Set index
howe_tows_transformed.set_index("FID", inplace=True)
howe_tows_transformed.head()

,snow_emergency_name,date,address,latitude,longitude,day_id,ward,neighborhood,community,tow_district
FID,,,,,,,,,,
1,Howe,2018-04-16 14:43:00,"710 RUSSELL AVE N,Minneapolis,MN",44.985471,-93.310768,2,5,Willard - Hay,Near North,1
2,Howe,2018-04-16 14:43:00,"150 26th Ave SE,Minneapolis,MN",44.971199,-93.220151,2,2,Prospect Park - East River Road,University,4
3,Howe,2018-04-16 14:44:00,"360 18th Ave NE,Minneapolis,MN",45.006359,-93.262426,2,1,Holland,Northeast,2
4,Howe,2018-04-16 14:46:00,"3520 31st st e ,Minneapolis,MN",44.939345,-93.227075,2,12,Howe,Longfellow,6
5,Howe,2018-04-16 14:47:00,"4004 Emerson Ave n,Minneapolis,MN",45.027867,-93.294223,2,4,Webber - Camden,Camden,1


### Connect to local database

In [5]:
rds_connection_string = 'mysql://root@localhost:3306/snow_emergency_db'
engine = create_engine(rds_connection_string)

### Use pandas to load csv converted DataFrame into database

In [ ]:
htows_df.to_sql(name='howe_tows_base',con=engine,if_exists='append',index=False)